<a href="https://colab.research.google.com/github/elhamod/IS883/blob/main/Prompt_Engineering_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Recap of How to use OpenAI API for chat: [GitHub](https://github.com/elhamod/IS883/blob/main/Open_Api_Guide.ipynb)

In [ ]:
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
config_ini_location = '/content/drive/MyDrive/Colab Notebooks/IS883/OpenAI guide/config.ini' # Change this to point to the location of your config.ini file.

import configparser

config = configparser.ConfigParser()
config.read(config_ini_location)
openai_api_key = config['OpenAI']['API_KEY']

In [ ]:
import openai

# Initialize the OpenAI API with your API key
openai.api_key = openai_api_key

# You can set up your API key by harcdcoding it here. It is a hacky and bad practice as others will see your secret key clearly and use your account. But, can be used for trying something quick and dirty
# openai.api_key = 'YOUR_OPENAI_API_KEY'

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="Translate the following English text to Spanish: 'Hello, how are you?'",
  max_tokens=50
)

print(response.choices[0].text.strip())

#Langchain Intro

Now, let's use another package, [`langchain`](https://python.langchain.com/docs/get_started/introduction), that uses OpenAI API and allows for more advanced capabilities.

In [ ]:
!pip install langchain

In [ ]:
from langchain.chat_models import ChatOpenAI

Let's create the model

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key) #temperature=0.0, model=llm_model

Let's create the prompt "template" to translate from English to Spanish.

In [ ]:
from langchain.prompts import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

Let's define the style of the translation

In [ ]:
customer_style = """informal Spanish"""

Let's query!

In [ ]:
customer_email = """
Hello, how are you?
"""


customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


print("Message: ", customer_messages)


customer_response = chat(customer_messages)


print("Response: ", customer_response.content)

## Extracting output from response

Let's say I want to get some information about a travel essay and extract specific variables from it.

In [ ]:
travel_template = """\
For the following text, extract the following information:

source: Where I am travelling from.

destination: Where I am travelling to.

airline: Which airline I am travelling with.

date: The date of travel in the format mm/dd/yyyy.

price: How much I paid for the ticket.

Format the output as JSON

text: {text}
"""

prompt_template = ChatPromptTemplate.from_template(travel_template)

In [ ]:
essay = """\
On Veteran's day, I woke up reluctantly in my bed in Boston. I did not want to go to work.\
I decided to give myself time off and go on a vacation. I went on Expedia and bought a ticket to \
Houston to see my family. I took the plane, arrived safely, and spent a great logn weekend with my cousins. \
It was the best $500 I had spent in a while. The only downside is that JetBlue has lost my luggage. :(
"""

Query!

In [ ]:
messages = prompt_template.format_messages(text=essay)
chat = ChatOpenAI(openai_api_key=openai_api_key)
response = chat(messages)
print(response)

Extract the information.

In [ ]:
import json

json_object = json.loads(response.content)
json_object['source']

#Using roles in OpenAI API

Roles provide context to the model to answer properly and more specifically.

- System role: defines that overarching context of the conversation.
- user role: The human agent.
- AI role: The AI bot.

In [ ]:
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

# System role
template = (
    "You are a teacher that is explaining advanced computer science concepts to someone in {degree}"
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# Human role
human_template = "Explain to me what {concept} is."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)




In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
print(chat(
    chat_prompt.format_prompt(
        degree="Computer science graduate program", concept="RNN"
    ).to_messages()
).content)

#A conversation with AI

We will use the memory capbility.



In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts import MessagesPlaceholder


In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

memory = ConversationBufferMemory()

# Defines a system message
template =  "You are my buttler Alfred. You are talking to Batman"
system_message = SystemMessagePromptTemplate.from_template(template=template)
PROMPT = PromptTemplate(input_variables=['history', 'input'], template=template + '.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

# Defines a conversation
conversation = ConversationChain(
    llm=chat,
    prompt=PROMPT,
    verbose=True,
    memory=memory,
)


In [ ]:
conversation.prompt

In [ ]:
memory

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
conversation.predict(input="How old am I?")

In [ ]:
conversation.predict(input="No, my age is 39")

In [ ]:
print(memory.buffer)

In [ ]:
conversation.predict(input="When was I born?")

In [ ]:
conversation.predict(input="Who is my son?")

In [ ]:
memory.buffer

For more debugging, wrap with `get_openai_callback()`

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    print(conversation.predict(input="Where are you now?"))
    print(cb)

In [ ]:
with get_openai_callback() as cb:
    print(conversation.predict(input="Write a poem about the adventurs of Batman and Joker in 400 words"))
    print(cb)

What if I want to limit the buffer to a window of 3 exchanges?

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

memory = ConversationBufferWindowMemory(k=10)

template =  "You are my buttler Alfred. You are talking to Batman"
system_message = SystemMessagePromptTemplate.from_template(template=template)
PROMPT = PromptTemplate(input_variables=['history', 'input'], template=template + '.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

conversation = ConversationChain(
    llm=chat,
    prompt=PROMPT,
    verbose=True,
    memory=memory,
)

In [ ]:
print(conversation.predict(input="What's my name?"))

In [ ]:
print(conversation.predict(input="Where am I?"))

In [ ]:
print(conversation.predict(input="I am now in Metropolis."))

In [ ]:
print(conversation.predict(input="Can you send me Robin?"))

In [ ]:
print(conversation.predict(input="Where am I?"))

##Sequential chains

We want to chain questions (The output of a question will be the input of another question)

![picture](https://miro.medium.com/v2/resize:fit:828/format:webp/1*hdx24fJuQwWm1fT-ULGQhg.jpeg)

In [ ]:
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Give me the names of 10 different countries in {input}"
)

# Chain 1
chain_one = LLMChain(llm=chat, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Order the names in this by country size: {input}"
)
# chain 2
chain_two = LLMChain(llm=chat, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                            #  verbose=True
                                            )
continent = "North America"
overall_simple_chain.run(continent)

Repeat the previous copde multiple times and see if there is anythin interesting.

Here is a somewhat more complicated chain:

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

In [ ]:
problem = "\
John has some amount of apples, Sarah has double that amount, \
and Mohannad has 3 apples. If they altogether have 12 apples, how man y does John have?\
"

In [ ]:
first_prompt = ChatPromptTemplate.from_template(
    "Convert the following problem into an equation in terms of x, where x is the number of apples John has:"
    "\n\n{problem}. Only give an equation"
)
chain_one = LLMChain(llm=chat, prompt=first_prompt,
                     output_key="Equation", verbose=True
                    )

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "After solving the following equation in terms of x:"
    "\n\n{Equation}, only provide the response as `x=`"
)
chain_two = LLMChain(llm=chat, prompt=second_prompt,
                     output_key="Solution", verbose=True
                    )


In [ ]:
third_prompt = ChatPromptTemplate.from_template(
    "Now, narrate how you solved this {problem} to a 6 year old"
)
chain_three = LLMChain(llm=chat, prompt=third_prompt,
                     output_key="narration"
                    )

In [ ]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["problem"],
    output_variables=["Equation", "Solution", "narration"],
    verbose=True
)

In [ ]:
import langchain
langchain.debug = True # Useful for debugging the stages of the chain

overall_chain(problem)

##Interacting with external data

Let's load a PDF and summarize the first page

In [ ]:
!pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/MohannadCV.pdf")
pages = loader.load_and_split()

In [ ]:
p = pages[0]

In [ ]:
m = "Read the following page and summarize it in 50 words: "+ p.page_content

In [ ]:
m

In [ ]:
langchain.debug = False

chat = ChatOpenAI(openai_api_key=openai_api_key)
chat.predict(m)

#Agents!

In [ ]:
!pip install -U wikipedia

Let's ask a question about GPT4 in Wikipedia

In [ ]:
from langchain.agents import load_tools, initialize_agent

llm = ChatOpenAI(openai_api_key=openai_api_key) # temperature=0,  #model=llm_model
tools = load_tools(["wikipedia"], llm=llm) #,"llm-math"

agent= initialize_agent(
    tools,
    llm,
    verbose = True)

msg = "When was ChatGPT 4 released?"

agent(msg)

What if we don't use Wikipedia agent?

In [ ]:
from langchain.schema import HumanMessage

msgs = [HumanMessage(content=msg)]
llm(msgs).content

###We can also define our own agent.

Define your own tool. The descriprion is what really tells the agent which tool to use.

In [ ]:
#!pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
agent= initialize_agent(
    [time], #tools +
    llm,
    # agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
agent("I ate a date. Will I be OK?")